In [1]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd
import os

import pickle
import multiprocessing

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
get_ipython().magic('matplotlib inline')

# machine learning
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier

#Evalaluation
from sklearn import metrics
from sklearn.cross_validation import cross_val_score

# Grid
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.feature_selection import RFE
from sklearn.model_selection import GridSearchCV

#Performance
%load_ext memory_profiler

D:\WindowsApps\Anaconda\envs\tensorflow\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Load

In [2]:
#os.chdir(r"D:/My Computer/")
#os.getcwd()

In [9]:
#Titanic
path = r"C:/Users/Nicol/Google Drive/Learning/Jupyter/Titanic"
#train_df = pd.read_csv(open(os.path.join(path, "clean_train.csv"), "r")) 
#test_df = pd.read_csv(open(os.path.join(path, "clean_test.csv"), "r"))

train_df = pd.read_csv(open(os.path.join(path, "clean_train2.csv"), "r")) 
test_df = pd.read_csv(open(os.path.join(path, "clean_test2.csv"), "r")) 

X = train_df.drop(["Survived"] , axis=1)
y = train_df["Survived"]

oosample  = test_df.drop(["PassengerId"] , axis=1).copy()
print(X.shape, y.shape, oosample.shape)

results=[]
def save(model, modelname):
    model.fit(X, y)
    submission = model.predict(oosample)
    df = pd.DataFrame({'PassengerId':test_df.PassengerId, 
                           'Survived':submission})
    df.to_csv((os.path.join(path,("submissions/{}.csv".format(modelname)))),header=True,index=False)
    
    # CV and Save Scores
    trainingscore = (grid.best_score_*100)
    results.append([(trainingscore),("{}".format(modelname)), grid.best_estimator_])
    print(trainingscore)
    print(grid.best_params_)

    with open((os.path.join(path,(r"Pickle/{}.pickle".format(modelname)))), 'wb') as f: pickle.dump(model, f)
        
def norm_save(model, modelname):
    model.fit(X, y)
    submission = model.predict(oosample)
    df = pd.DataFrame({'PassengerId':test_df.PassengerId, 
                           'Survived':submission})
    df.to_csv((os.path.join(path,("submissions/{}.csv".format(modelname)))),header=True,index=False)
    with open((os.path.join(path,(r"Pickle/{}.pickle".format(modelname)))), 'wb') as f: pickle.dump(model, f)

def ensembling(model, modelname):
    model.fit(X, y)
    submission = model.predict(oosample)
    df = pd.DataFrame({'PassengerId':test_df.PassengerId, 
                           'Survived':submission})
    print(len(df))
    df.to_csv((os.path.join(path,("submissions/{}.csv".format(modelname)))),header=True,index=False)
    with open((os.path.join(path,(r"Pickle/{}.pickle".format(modelname)))), 'wb') as f: pickle.dump(model, f)

(891, 11) (891,) (418, 11)


In [3]:
print(y.value_counts(normalize=True))

0    0.616162
1    0.383838
Name: Survived, dtype: float64


In [4]:
# use train/test split with different random_state values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2017)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

# Stratified Cross Validation
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.3, random_state=42)

# Normal CV
#cv =5

In [5]:
print(X.info())
print(y.head())
print(oosample.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
Pclass         891 non-null int64
Sex            891 non-null int64
Age            891 non-null int64
Parch          891 non-null int64
Fare           891 non-null int64
Embarked       891 non-null int64
Name_length    891 non-null int64
Has_Cabin      891 non-null int64
FamilySize     891 non-null int64
IsAlone        891 non-null int64
Title          891 non-null int64
dtypes: int64(11)
memory usage: 76.6 KB
None
0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
Pclass         418 non-null int64
Sex            418 non-null int64
Age            418 non-null int64
Parch          418 non-null int64
Fare           418 non-null int64
Embarked       418 non-null int64
Name_length    418 non-null int64
Has_Cabin      418 non-null int64
FamilySize     418 non-null int64
I

In [6]:
# import seaborn as sns; sns.set()
# sns.pairplot(train_df, hue='Survived', size=1.5);

# Generative Classification
Probabilistically determine the label for a new point

## Gaussian

In [7]:
?GaussianNB

In [11]:
%%time
%%memit
model = GaussianNB()

score = cross_val_score(model, X, y, cv=10, scoring='accuracy')
print(score.mean())
norm_save(GaussianNB(), "Gaussian")

0.779046929974
0.779046929974
0.779046929974
peak memory: 132.99 MiB, increment: 0.01 MiB
Wall time: 1.58 s


## Logistic


In [9]:
?LogisticRegression

In [13]:
%%memit
model= LogisticRegression()
score = cross_val_score(model, X, y, cv=10, scoring='accuracy')
print(score.mean())
norm_save(LogisticRegression(), "Logistic_Regression")

0.797933832709
0.797933832709
peak memory: 133.41 MiB, increment: 0.42 MiB


# Non-Parametric

# Esemble Method

Means that a bunch of the model get created and are aggregated at the end for best performance.

## Bagging

Aka Bootstrap- creates a bunch of trees using a random 3/4 the the data for each, while using sampling without replacement, which means that values may be sampled multiple times.

https://www.analyticsvidhya.com/blog/2015/06/tuning-random-forest-model/
HyperParameters:
- max_features: This is the random subset of features to be used for splitting node, the lower the better to reduce variance. For Classification model, ideal max_features = sqr(n_var)
- n_estimators: # of trees built before average prediciton is made
- min_sample_leaf: End node of trees. Too small = more noise. For Regression tree.
- n_jobs: computer processors utilized. -1 = no restrictions
- random_state: seed()

In [11]:
tree = DecisionTreeClassifier()
bag = BaggingClassifier(tree, n_estimators=300, max_samples=0.8,
                        random_state=1)

print(cross_val_score(bag, X, y, cv=10, scoring='accuracy').mean()*100)

81.2705708773


In [12]:
%%memit
param_grid ={'n_estimators': np.arange(20, 500, 25)}

tree = DecisionTreeClassifier()
#bag = BaggingClassifier(tree)

grid = GridSearchCV(BaggingClassifier(tree),
                    param_grid, cv=cv, scoring='accuracy',
                    verbose=1)

grid.fit(X, y)
save(grid.best_estimator_, "Bagger_ensemble")

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   30.0s finished


78.6567164179
{'n_estimators': 445}
peak memory: 144.98 MiB, increment: 11.08 MiB


## Random Forest

Trees are created wih a randomly picked subset of observations and variables. More uncorrelated splits, less overemphasis on certain features.

In [13]:
%%memit
model = RandomForestClassifier(n_estimators=500)
model.fit(X_train, y_train)
print(model.score(X_test, y_test))

0.810055865922
peak memory: 156.43 MiB, increment: 11.45 MiB


In [14]:
RandomForestClassifier().get_params().keys()

dict_keys(['class_weight', 'max_leaf_nodes', 'random_state', 'min_impurity_split', 'max_features', 'n_jobs', 'verbose', 'warm_start', 'min_samples_leaf', 'n_estimators', 'min_weight_fraction_leaf', 'min_samples_split', 'max_depth', 'bootstrap', 'oob_score', 'criterion'])

In [15]:
%%memit
param_grid ={'max_depth': np.arange(6, 11, 1),
             'n_estimators':np.arange(350, 450, 25),
             'max_features':np.arange(0.5,.81, 0.05),
            'max_leaf_nodes':np.arange(6, 10, 1)}
#param_grid ={'n_estimators':[200]}

from sklearn import feature_selection

#model = feature_selection.RFE(RandomForestClassifier())
model= RandomForestClassifier()

grid = GridSearchCV(model,
                    param_grid, cv=cv,
                    scoring='accuracy',
                    verbose=1)

grid.fit(X, y)
save(grid.best_estimator_, "Random_Forest")

Fitting 5 folds for each of 560 candidates, totalling 2800 fits


[Parallel(n_jobs=1)]: Done 2800 out of 2800 | elapsed: 16.4min finished


81.1940298507
{'max_leaf_nodes': 8, 'max_depth': 7, 'n_estimators': 425, 'max_features': 0.55000000000000004}
peak memory: 156.43 MiB, increment: 0.00 MiB


## Extremely Randomized Trees (ExtraTree)

## AdaBoostClassifier

Method, similarly to deep learning, applies weights to all data points and optimizes them using the loss function. Fixes mistakes by assigning high weights to them during iterative process.

In [18]:
#?AdaBoostClassifier

In [19]:
%%memit
param_grid ={'n_estimators':np.arange(50, 301, 25),
            'learning_rate':np.arange(.1, 4, .5)}

grid = GridSearchCV(AdaBoostClassifier(),
                    param_grid,cv=cv, scoring='accuracy',
                    verbose=1)

grid.fit(X, y);
save(grid.best_estimator_, "AdaBoost_Ensemble")

Fitting 5 folds for each of 88 candidates, totalling 440 fits


[Parallel(n_jobs=1)]: Done 440 out of 440 | elapsed:  1.2min finished


80.5223880597
{'learning_rate': 0.10000000000000001, 'n_estimators': 150}
peak memory: 148.86 MiB, increment: 0.00 MiB


## Gradient Boosting Classifier

GB builds an additive model in a forward stage-wise fashion; it allows for the optimization of arbitrary differentiable loss functions.

Part of the generalized boosting algorithms. Can use more loss functions than AdaBoost, and uses gradients instead of high-weight data points.

In [16]:
#?GradientBoostingClassifier

In [17]:
%%memit
param_grid ={'n_estimators':np.arange(100, 301, 25),
            'loss': ['deviance', 'exponential'],
            'learning_rate':np.arange(0.01, 0.32,.05),
            'max_depth': np.arange(2, 4.1, .5)}

grid = GridSearchCV(GradientBoostingClassifier(),
                    param_grid,cv=cv,
                    scoring='accuracy',
                    verbose=1)

grid.fit(X, y)
save(grid.best_estimator_, "Gradient_Boosting")

Fitting 5 folds for each of 630 candidates, totalling 3150 fits
81.3432835821
{'learning_rate': 0.060000000000000005, 'max_depth': 2.0, 'loss': 'deviance', 'n_estimators': 100}
peak memory: 148.86 MiB, increment: 2.79 MiB


[Parallel(n_jobs=1)]: Done 3150 out of 3150 | elapsed:  4.6min finished


## eXtreme Gradient Boosting

## KNN

In [21]:
#?KNeighborsClassifier

In [22]:
%%memit
param_grid ={'n_neighbors': np.arange(1,21,1),
            'weights':['uniform','distance']
            }

grid = GridSearchCV(KNeighborsClassifier(),
                    param_grid,cv=cv, scoring='accuracy',
                    verbose=1)

grid.fit(X, y)
save(grid.best_estimator_, "KNN")

Fitting 5 folds for each of 40 candidates, totalling 200 fits
77.3134328358
{'n_neighbors': 5, 'weights': 'distance'}
peak memory: 149.38 MiB, increment: 0.52 MiB


[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    1.4s finished


# Discriminative Classification
Model new points by seeing where it falls upon a divide.
Fast prediction phase, work well in high dimensional data, versatile

Costly at high quantities of data

### Stochastic Gradient Descent

In [23]:
?SGDClassifier

In [24]:
%%memit
param_grid ={'loss':["hinge","log","modified_huber","squared_hinge","epsilon_insensitive","squared_epsilon_insensitive"]
            }

grid = GridSearchCV(SGDClassifier(),
                    param_grid,cv=cv, scoring='accuracy',
                    verbose=1)

grid.fit(X, y)
save(grid.best_estimator_, "StochasticGradientDescent")

Fitting 5 folds for each of 6 candidates, totalling 30 fits
65.223880597
{'loss': 'hinge'}


[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.0s finished


Fitting 5 folds for each of 6 candidates, totalling 30 fits
71.2686567164
{'loss': 'modified_huber'}
peak memory: 149.43 MiB, increment: 0.04 MiB


[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.0s finished


In [25]:
?GridSearchCV

## Support Vector Classifier
Creates a linear divide between point to classify. Maximizes the distance of the discriminatory margin.

Hyperparameters:
- C: Hardness of the margin. Higher C, less softening.


Radial Basis Function (RBF)
- Gamma: how far the influence of a single training example raches. low=far, high=close, Inverse of the radius of influence of samples selected by the model as support vectors.

In [26]:
%%memit

# Define Model
model = svm.LinearSVC()
#Fit Model
scores= cross_val_score(model, X, y, cv=10, scoring='accuracy')
print(scores.mean()*100)

#submit(svm.LinearSVC(), name="80linear_svc.csv")

78.3327658609
peak memory: 149.43 MiB, increment: 0.01 MiB


### RBF SVC

In [ ]:
%%memit
param_grid = [
  {'C': np.arange(25,176,5), 'gamma': np.logspace(1, -4, 10), 'kernel': ['rbf'],"probability" : [True]}
 ]

cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
grid = GridSearchCV(svm.SVC(),
                    param_grid, cv=cv,
                    scoring='accuracy', verbose=1)

grid.fit(X, y)
save(grid.best_estimator_, "SVCrbf")

Fitting 5 folds for each of 310 candidates, totalling 1550 fits


[Parallel(n_jobs=1)]: Done 1550 out of 1550 | elapsed:  2.2min finished


80.4469273743
{'gamma': 0.016681005372000592, 'kernel': 'rbf', 'probability': True, 'C': 65}
peak memory: 152.59 MiB, increment: 3.16 MiB


### Linear SVC

In [14]:
%%memit
param_grid = {'C': [1,10],'kernel':['linear'], "probability" : [True]}

grid = GridSearchCV(svm.SVC(),
                    param_grid, cv=cv,
                    scoring='accuracy', verbose=1)

grid.fit(X, y)
save(grid.best_estimator_, "SVCLinear")

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   30.0s finished


78.8805970149
{'kernel': 'linear', 'C': 1, 'probability': True}
peak memory: 135.85 MiB, increment: 2.44 MiB


## PCA + SVC Pipeline

In [8]:
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline

pca = PCA(n_components=5, whiten=True, random_state=42, svd_solver='randomized')
svc = SVC(kernel='rbf', class_weight='balanced')
model = make_pipeline(pca, svc)
# pipeline!

param_grid = [{'svc__C': np.logspace(-2, 3, 6),
              'svc__gamma': np.logspace(1, -7, 10)},
              {'svc__C': [0.001, 0.01, 0.1, 1, 10, 50, 100, 150, 1000, 1500], 'svc__kernel': ['linear']}]
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
grid = GridSearchCV(model, param_grid, cv=cv, verbose=1)

grid.fit(X, y)
save(grid.best_estimator_, "PCA_SVC")

Fitting 5 folds for each of 70 candidates, totalling 350 fits
80.8938547486
{'svc__C': 1.0, 'svc__gamma': 0.1668100537200059}


[Parallel(n_jobs=1)]: Done 350 out of 350 | elapsed:   19.5s finished


## Results

In [9]:
import csv
with open(os.path.join(path, "results.csv"), 'w') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(results)

In [10]:
results = pd.read_csv(open(os.path.join(path, "results.csv"), "r")).transpose()

In [11]:
results

""
"[80.89385474860336, 'PCA_SVC', Pipeline(steps=[('pca', PCA(copy=True, iterated_power='auto', n_components=5, random_state=42,\n svd_solver='randomized', tol=0.0, whiten=True)), ('svc', SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,\n decision_function_shape=None, degree=3, gamma=0.1668100537200059,\n kernel='rbf', max_iter=-1, probability=False, random_state=None,\n shrinking=True, tol=0.001, verbose=False))])]"


## MODEL ENSEMBLE

In [25]:
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
import numpy as np
import pickle

# set seed for reproducability
np.random.seed(2017)

import statsmodels.api as sm
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier

# currently its available as part of mlxtend and not sklearn
from mlxtend.classifier import EnsembleVoteClassifier
from sklearn import cross_validation
from sklearn import metrics
from sklearn.cross_validation import train_test_split

D:\WindowsApps\Anaconda\envs\tensorflow\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [26]:
open_file = open(os.path.join(path,"Pickle/AdaBoost_Ensemble.pickle"), "rb")
Ada_Boost_Ensemble = pickle.load(open_file)
open_file.close()

print(Ada_Boost_Ensemble)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=0.10000000000000001, n_estimators=150,
          random_state=None)


In [27]:
open_file = open(os.path.join(path,"Pickle/Logistic_Regression.pickle"), "rb")
Logistic_Regression = pickle.load(open_file)
open_file.close()    

open_file = open(os.path.join(path,"Pickle/Random_Forest.pickle"), "rb")
Random_Forest = pickle.load(open_file)
open_file.close()

open_file = open(os.path.join(path,"Pickle/SVCrbf.pickle"), "rb")
SVCrbf = pickle.load(open_file)
open_file.close()

open_file = open(os.path.join(path,"Pickle/SVCLinear.pickle"), "rb")
SVCLinear = pickle.load(open_file)
open_file.close()

open_file = open(os.path.join(path,"Pickle/KNN.pickle"), "rb")
KNN = pickle.load(open_file)
open_file.close()

open_file = open(os.path.join(path,"Pickle/Gradient_Boosting.pickle"), "rb")
Gradient_Boosting = pickle.load(open_file)
open_file.close()

open_file = open(os.path.join(path,"Pickle/AdaBoost_Ensemble.pickle"), "rb")
Ada_Boost_Ensemble = pickle.load(open_file)
open_file.close()

open_file = open(os.path.join(path,"Pickle/Bagger_ensemble.pickle"), "rb")
Bagger_ensemble = pickle.load(open_file)
open_file.close()

open_file = open(os.path.join(path,"Pickle/StochasticGradientDescent.pickle"), "rb")
StochasticGradientDescent = pickle.load(open_file)
open_file.close()

open_file = open(os.path.join(path,"Pickle/Gaussian.pickle"), "rb")
Gaussian = pickle.load(open_file)
open_file.close()

open_file = open(os.path.join(path,"Pickle/PCA_SVC.pickle"), "rb")
PCA_SVC = pickle.load(open_file)
open_file.close()

In [28]:
X_train[:5]

,Pclass,Sex,Age,Parch,Fare,Embarked,Name_length,Has_Cabin,FamilySize,IsAlone,Title
304,3,1,1,0,1,0,33,0,1,1,1
149,2,1,2,0,1,0,33,0,1,1,5
484,1,1,1,0,3,1,23,1,2,0,1
430,1,1,1,0,2,0,41,1,1,1,1
722,2,1,2,0,1,0,28,0,1,1,1


In [29]:
LR = Logistic_Regression
RF = Random_Forest
SVMR = SVCrbf
SVML = SVCLinear
KNC = KNN
GBC = Gradient_Boosting
ABC = Ada_Boost_Ensemble
BC = Bagger_ensemble
GAU = Gaussian
SGD = StochasticGradientDescent
PCA_SVC_LI = PCA_SVC

models= list(zip([LR,
                  RF,
                  SVMR,
                  SVCLinear,
                  KNC,
                  GBC, 
                  ABC,
                  BC,
                  PCA_SVC_LI,
                  GAU,
                  SGD], 
                      ['Logistic Regression', 
                       'Random Forest', 
                       'RBG Support Vector Classifier',
                       'Linear Support Vector Classifier'
                       'KNeighbors',
                       'Gradient Boosting',
                       'Ada Boost',
                       'Bagging',
                       'Principle Component Linear SVC',
                       'Gaussian',
                       'Stochastic Gradient Descent']))

In [30]:
clfs = []
print('5-fold cross validation:\n')
for clf, label in models:
    scores = cross_validation.cross_val_score(clf, X_train, y_train,cv=5, scoring='accuracy')
    print("Train CV Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))
    md = clf.fit(X, y)    
    clfs.append(md)
    print("Test Accuracy: %0.2f " % (metrics.accuracy_score(clf.predict(X_test), y_test)))

5-fold cross validation:

Train CV Accuracy: 0.79 (+/- 0.02) [Logistic Regression]
Test Accuracy: 0.81 
Train CV Accuracy: 0.82 (+/- 0.01) [Random Forest]
Test Accuracy: 0.87 
Train CV Accuracy: 0.80 (+/- 0.01) [RBG Support Vector Classifier]
Test Accuracy: 0.89 
Train CV Accuracy: 0.80 (+/- 0.02) [Linear Support Vector ClassifierKNeighbors]
Test Accuracy: 0.84 
Train CV Accuracy: 0.76 (+/- 0.02) [Gradient Boosting]
Test Accuracy: 0.99 
Train CV Accuracy: 0.81 (+/- 0.02) [Ada Boost]
Test Accuracy: 0.87 
Train CV Accuracy: 0.81 (+/- 0.02) [Bagging]
Test Accuracy: 0.86 
Train CV Accuracy: 0.78 (+/- 0.03) [Principle Component Linear SVC]
Test Accuracy: 0.98 
Train CV Accuracy: 0.81 (+/- 0.01) [Gaussian]
Test Accuracy: 0.85 
Train CV Accuracy: 0.78 (+/- 0.03) [Stochastic Gradient Descent]
Test Accuracy: 0.80 


## Voting

Hard- Mode
Soft- Probabilistic

In [31]:
# ### Ensemble Voting
print('5-fold cross validation:\n')

# [LR, RF, SVM, KNC,GBC,ABC, BC]

allmodel = [LR, RF, SVMR, SVCLinear, KNC, GBC, ABC, BC, PCA_SVC_LI, GAU]
best= [LR, RF, SVMR, SVCLinear, KNC, GBC, ABC, BC, GAU]

ECH = EnsembleVoteClassifier(allmodel, voting='hard')
ECS = EnsembleVoteClassifier(best, voting='soft', weights=[1,2,2,1,2,1,1,1,1])

for clf, label in zip([ECH, ECS], 
                      ['Ensemble Hard Voting', 
                       'Ensemble Soft Voting']):
    scores = cross_validation.cross_val_score(clf, X_train, y_train, cv=5, scoring='accuracy')
    print("Train CV Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))
    md = clf.fit(X, y)    
    clfs.append(md)
    print("Test Accuracy: %0.2f " % (metrics.accuracy_score(clf.predict(X_test), y_test)))

5-fold cross validation:

Train CV Accuracy: 0.82 (+/- 0.01) [Ensemble Hard Voting]
Test Accuracy: 0.88 
Train CV Accuracy: 0.81 (+/- 0.03) [Ensemble Soft Voting]
Test Accuracy: 0.94 


In [32]:
ensembling(ECH,"Hard_ensemble1")
ensembling(ECS,"Soft_ensemble1")

418
418


## Stacking
Stacked generalized modesl

In [35]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
from sklearn import cross_validation
from sklearn import metrics

from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
%matplotlib inline

# Classifiers
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

np.random.seed(2017)  # seed to shuffle the train set

X = train_df.drop(["Survived"] , axis=1)
y = train_df["Survived"]

oosample  = test_df.drop(["PassengerId"] , axis=1).copy()
print(X.shape, y.shape, oosample.shape)

#Normalize
X = StandardScaler().fit_transform(X)

# evaluate the model by splitting into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2017)

kfold = cross_validation.StratifiedKFold(y=y_train, n_folds=5, random_state=2017)
num_trees = 10
verbose = True # to print the progress

clfs = [KNeighborsClassifier(),
        RandomForestClassifier(n_estimators=num_trees, random_state=2017),
        GradientBoostingClassifier(n_estimators=num_trees, random_state=2017)]

# Creating train and test sets for blending
dataset_blend_train = np.zeros((X_train.shape[0], len(clfs)))
dataset_blend_test = np.zeros((X_test.shape[0], len(clfs)))

print('5-fold cross validation:\n')
for i, clf in enumerate(clfs):   
    scores = cross_validation.cross_val_score(clf, X_train, y_train, cv=kfold, scoring='accuracy')
    print("##### Base Model %0.0f #####" % i)
    print("Train CV Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))
    clf.fit(X_train, y_train)   
    print("Train Accuracy: %0.2f " % (metrics.accuracy_score(clf.predict(X_train), y_train)))
    dataset_blend_train[:,i] = clf.predict_proba(X_train)[:, 1]
    dataset_blend_test[:,i] = clf.predict_proba(X_test)[:, 1]
    print("Test Accuracy: %0.2f " % (metrics.accuracy_score(clf.predict(X_test), y_test)))    

print("##### Meta Model #####")
clf = LogisticRegression()
scores = cross_validation.cross_val_score(clf, dataset_blend_train, y_train, cv=kfold, scoring='accuracy')
clf.fit(dataset_blend_train, y_train)
print("Train CV Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))
print("Train Accuracy: %0.2f " % (metrics.accuracy_score(clf.predict(dataset_blend_train), y_train)))
print("Test Accuracy: %0.2f " % (metrics.accuracy_score(clf.predict(dataset_blend_test), y_test)))

(891, 11) (891,) (418, 11)
5-fold cross validation:

##### Base Model 0 #####
Train CV Accuracy: 0.80 (+/- 0.02)
Train Accuracy: 0.86 
Test Accuracy: 0.88 
##### Base Model 1 #####
Train CV Accuracy: 0.80 (+/- 0.03)
Train Accuracy: 0.95 
Test Accuracy: 0.83 
##### Base Model 2 #####
Train CV Accuracy: 0.81 (+/- 0.02)
Train Accuracy: 0.83 
Test Accuracy: 0.87 
##### Meta Model #####
Train CV Accuracy: 0.96 (+/- 0.02)
Train Accuracy: 0.96 
Test Accuracy: 0.82 


## Reflection on model

Ensemble models suggest that testing accuracy is in the high 80s, however when applied to Kaggle out of sample data, perfromance is consistently in the high 70s. This suggests that either the submission data is very different, or that my model is overfitting on the given data.

 - May want to explore models with a greater emphasis on randomness in order to tone down the overfitting.
 - Perhaps compare variable distribution between submission data and training data